3.	Необходимо написать на базе BERT определитель жанра фильма: 
a.	На вход подается сюжет фильма. На выходе - жанр 
sample: https://www.kaggle.com/balraj98/movie-genre-prediction-from-wiki-plot-using-bert 